In [10]:
import json
from urllib.request import urlopen
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math
import numpy as np
from plotly.subplots import make_subplots

In [12]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [13]:
med_inc = pd.read_csv("census_2020_median_inc_and_poverty.csv", dtype={'state': object, 'fips_county': object})
med_inc

,Unnamed: 0,state_abbrev,med_inc,pov_rate,pov_count,county,time,state,fips_county
0,0,AL,67565.0,11.2,6242.0,Autauga County,2020,1,001
1,1,AL,71135.0,8.9,20189.0,Baldwin County,2020,1,003
2,2,AL,38866.0,25.5,5548.0,Barbour County,2020,1,005
3,3,AL,50907.0,17.8,3549.0,Bibb County,2020,1,007
4,4,AL,55203.0,13.1,7525.0,Blount County,2020,1,009
...,...,...,...,...,...,...,...,...,...
3138,3138,WY,70583.0,7.6,3187.0,Sweetwater County,2020,56,037
3139,3139,WY,92488.0,5.2,1207.0,Teton County,2020,56,039
3140,3140,WY,71246.0,8.5,1697.0,Uinta County,2020,56,041
3141,3141,WY,58532.0,9.7,739.0,Washakie County,2020,56,043


In [30]:


zhvi = pd.read_csv("zillow_home_value_index_county.csv", index_col="RegionID")

med_inc_pov = pd.read_csv("census_2020_median_inc_and_poverty.csv", dtype={'state': object, 'fips_county': object})

crosswalk = pd.read_csv("CountyCrossWalk_Zillow.csv", encoding = "ISO-8859-1", index_col="CountyRegionID_Zillow", dtype={"FIPS" : object})

population = pd.read_csv("county_population.csv", encoding = "ISO-8859-1", dtype={"STATE" : object, "COUNTY" : object})

population["STATE"] = population["STATE"].str.zfill(2)
population["COUNTY"] = population["COUNTY"].str.zfill(3)
population["COUNTY_FIPS"] = population["STATE"] + population["COUNTY"]

med_inc_pov["state"] = med_inc['state'].str.zfill(2)
med_inc_pov['County Code'] = med_inc["state"] + med_inc["fips_county"]

years = zhvi.loc[:, "2019-01-31":"2021-12-31":1]

regions = zhvi.loc[:, "SizeRank":"MunicipalCodeFIPS":1]

zhvi2 = regions.join(years)

zhvi2["2019_average"] = zhvi2.loc[:, "2019-01-31":"2019-12-31": 1].mean(axis=1)
zhvi2["2020_average"] = zhvi2.loc[:, "2020-01-31":"2020-12-31": 1].mean(axis=1)
zhvi2["2021_average"] = zhvi2.loc[:, "2021-01-31":"2021-12-31": 1].mean(axis=1)
zhvi2["2020_increase"] = ((zhvi2["2020_average"] - zhvi2["2019_average"])/zhvi2["2019_average"])*100
zhvi2["2021_increase"] = ((zhvi2["2021_average"] - zhvi2["2020_average"])/zhvi2["2020_average"])*100
zhvi2["2021_2yr_increase"] = ((zhvi2["2021_average"] - zhvi2["2019_average"])/zhvi2["2019_average"])*100

zhvi2["text_20"] = " "
zhvi2["text_21"] = " "
zhvi2["text_2yrs"] = " "

zhvi2.dropna()

zhvi_county = pd.merge(zhvi2, crosswalk, left_index=True, right_index=True)
zhvi_county["FIPS"] = zhvi_county["FIPS"].str.zfill(5)
zhvi_county_inc = pd.merge(zhvi_county, med_inc_pov, left_on="FIPS", right_on="County Code")
zhvi_county_inc_pop = pd.merge(zhvi_county_inc, population, left_on="FIPS", right_on="COUNTY_FIPS")
zhvi_county_inc_pop


# Format text for plotting
for idx, row in zhvi_county_inc_pop.iterrows():
    zhvi_county_inc_pop.at[idx, 'text_20'] = 'County:' + row["RegionName"] + '<br>' + 'State:' + row["State"] + \
    '<br>' + '2019-20 increase:' + str(round(row["2020_increase"], 3)) + '<br>' + 'Med_Inc:' + str(row["med_inc"]) + \
        '<br>' + 'Pop_2019:' + str(row["POPESTIMATE2019"])
    
    zhvi_county_inc_pop.at[idx, 'text_21'] = 'County:' + row["RegionName"] + '<br>' + 'State:' + row["State"] + \
    '<br>' + '2020-21 increase:' + str(round(row["2021_increase"], 3)) + '<br>' + 'Med_Inc:' + str(row["med_inc"]) + \
        '<br>' + 'Pop_2020:' + str(row["POPESTIMATE2020"])
    
    zhvi_county_inc_pop.at[idx, 'text_2yrs'] = 'County:' + row["RegionName"] + '<br>' + 'State:' + row["State"] + \
    '<br>' + '2019-21 increase:' + str(round(row["2021_2yr_increase"], 3)) + '<br>' + 'Med_Inc:' + str(row["med_inc"]) + \
        '<br>' + 'Pop_2020:' + str(row["POPESTIMATE2020"])

#print(max(zhvi_county_inc_pop["2021_2yr_increase"]))
print(np.median(zhvi_county_inc_pop["med_inc"]))
#zhvi_county_inc_pop["med_inc"].value_counts()[np.median(zhvi_county_inc_pop["med_inc"])]

55786.5


In [27]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=zhvi_county_inc_pop["med_inc"]))
fig.add_shape(
        go.layout.Shape(type='line', xref='x', yref='paper',
                        x0=np.median(zhvi_county_inc_pop.med_inc), y0=0.0, x1=np.median(zhvi_county_inc_pop.med_inc),
                        y1=160.0,
                        line={'dash': 'dash'}))
#fig.add_vline(x=np.median(zhvi_county_inc_pop.med_inc), line_dash = 'dash', line_color = 'green')
#fig.add_vline(x=np.mean(zhvi_county_inc_pop.med_inc), line_dash = 'dash', line_color = 'firebrick')
fig.show()

In [9]:
fig = px.histogram(zhvi_county_inc_pop, x="pov_rate")
fig.add_vline(x=np.median(zhvi_county_inc_pop.pov_rate), line_dash = 'dash', line_color = 'green')
fig.add_vline(x=float(zhvi_county_inc_pop.loc[zhvi_county_inc_pop["FIPS"] == "30029", 'pov_rate']), line_dash = 'dash', line_color = 'firebrick')
fig.update_layout(title="Distribution of Poverty rate of counties")
fig.show()

In [16]:
def create_income_graph(zhvi_county_inc_pop, FIPS):
    '''
    Create county income distribution graph
    '''

    fig = make_subplots(rows=2, cols=1)

    fig.add_trace(px.histogram(zhvi_county_inc_pop, x="med_inc"))
    fig.add_vline(x=np.median(zhvi_county_inc_pop.med_inc), line_dash = 'dash', line_color = 'green')
    fig.add_vline(x=float(zhvi_county_inc_pop.loc[zhvi_county_inc_pop["FIPS"] == FIPS, 'med_inc']), line_dash = 'dash', line_color = 'firebrick')
    fig.update_layout(title="Distribution of Median Incomes of Counties <br> Red line = selected county")

    fig.add_trace(px.histogram(zhvi_county_inc_pop, x="pov_rate"))
    fig.add_vline(x=np.median(zhvi_county_inc_pop.pov_rate), line_dash = 'dash', line_color = 'green')
    fig.add_vline(x=float(zhvi_county_inc_pop.loc[zhvi_county_inc_pop["FIPS"] == "30029", 'pov_rate']), line_dash = 'dash', line_color = 'firebrick')
    fig.update_layout(title="Distribution of Poverty rate of counties")


    return fig

create_income_graph(zhvi_county_inc_pop, "30029")

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [Figure({
    'data': [{'alignmentgroup': 'True',
              'bingroup': 'x',
              'hovertemplate': 'med_inc=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'histogram',
              'x': array([71611., 61906., 70390., ..., 43367., 59409., 63461.]),
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'med_inc'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': '

In [ ]:
fig = go.Figure(go.Choroplethmapbox(geojson=counties, locations=zhvi_county_inc_pop["FIPS"], z=zhvi_county_inc_pop["2021_2yr_increase"],
                                    colorscale="Jet", zmin=-17, zmax=58, text=zhvi_county_inc_pop["text_2yrs"], 
                                    marker_opacity=0.5, marker_line_width=0))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()